In [1]:
!python pointer_generator/preprocess.py --source sample_data/train.src --vocab <(cut -d' ' -f1 sample_data/dict.pg.txt) --source-out sample_data/train.pg.src

In [1]:
batch_size=1
beam_size=6
max_length=100
length_penalty=1.0

!fairseq-interactive bin \
    --user-dir pointer_generator/pointer_generator_src \
    --batch-size "$batch_size" \
    --task translation \
    --source-lang src --target-lang tgt \
    --path checkpoints/checkpoint24.pt \
    --input sample_data/train.pg.src \
    --buffer-size 200 \
    --max-len-a 0 \
    --max-len-b "$max_length" \
    --lenpen "$length_penalty" \
    --beam "$beam_size" \
    --skip-invalid-size-inputs-valid-test | tee sample_data/generate.out

INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
DEBUG:hydra.core.utils:Setting JobRuntime:name=UNKNOWN_NAME
DEBUG:hydra.core.utils:Setting JobRuntime:name=utils
INFO:fairseq_cli.interactive:{'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': 'pointer_generator/pointer_generator_src', 'empty_cache_freq': 0, 'all_gather_list_size': 1

In [2]:
!grep ^H sample_data/generate.out | cut -f 3- > sample_data/generate.hyp

In [3]:
!python examples/pointer_generator/postprocess.py \
    --source <(awk 'NF<1024' sample_data/train.src) \
    --target sample_data/generate.hyp \
    --target-out generate.hyp.processed